# Lab 2: Thresholding and blob analysis

# Intro to Image Processing


##Step 1: Load the Dependencies

> This section loads some required libraries used in this notebook: **numpy**, **pandas**, **cv2**, **skimage**, **PIL**, **matplotlib**

*   [Numpy](https://www.numpy.org/) is an array manipulation library, used for linear algebra, Fourier transform, and random number capabilities.
*   [Pandas](https://pandas.pydata.org/) is a library for data manipulation and data analysis.
*   [CV2](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_image_display/py_image_display.html) is a library for computer vision tasks.
*   [Skimage](https://scikit-image.org/) is a library which supports image processing applications on python.
*   [Matplotlib](https://matplotlib.org/) is a library which generates figures and provides graphical user interface toolkit.









In [ ]:
import numpy as np
import pandas as pd
import cv2 as cv 
import os
from google.colab import files
from google.colab.patches import cv2_imshow # for image display
from skimage import io
from PIL import Image 
import matplotlib.pyplot as plt
import sys

##Step 2: Read Image from Urls

> In this step we will read images from urls, and display them using openCV, please note the difference when reading images in RGB and BGR format. The default input color channels are in BGR (Blue, Green, Red) format for openCV.

In [ ]:
# Create a list of urls of the images
urls = ["https://placekitten.com/800/500",
        "https://placekitten.com/600/400"]

# Read and display the images

for url in urls:
  # read image from url using skimage.io 
  image = io.imread(url) 

  # convert to RGB colorspace 
  image_2 = cv.cvtColor(image, cv.COLOR_BGR2RGB)

  # concantenate both images
  final_frame = cv.hconcat((image, image_2))

  # display images with OpenCV 
  cv2_imshow(final_frame)
  print('\n')

#### ToDo #1: Read an image from a URL and display it

Image source examples:

[Place Kitten](https://placekitten.com/) - use the base Place Kitten URL followed by a width and height separated by backslashes ''/''. For example, use the URL `https://placekitten.com/500/300` to fetch a cat image with a width of 500px and height of 300px.

[Google Image search](https://www.google.com/imghp?hl=en) - search for an image. Left-click one of the returned images, then right-click on the full image, and then select "Copy Image Address".

In [ ]:
## TODO: LOAD IMAGE
## url = 
## myImg = io.imread(url)
## myImg_rgb = cv.cvtColor(myImg, cv.COLOR_BGR2RGB)
## cv2_imshow(myImg_rgb)


##Step 3: Image Data and Histograms

In [ ]:
# wedge-tailed eagle photo
eagle_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/0/03/Wedge-tailed_Eagle_It_is_Not_Even_Playing_the_Game%21_%2850096251297%29.jpg/513px-Wedge-tailed_Eagle_It_is_Not_Even_Playing_the_Game%21_%2850096251297%29.jpg"

In [ ]:
# Load image of eagle
image = io.imread(eagle_url)

# convert to RGB
image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)

# concantenate both images
final_frame = cv.hconcat((image, image_rgb))

# Display the image
cv2_imshow(final_frame)

In [ ]:
# Check the image matrix data type 
print('dtype:', image_rgb.dtype)

# Check the image dimensions 
print('dimensions [w, h]:', [image_rgb.shape[1], image_rgb.shape[0]])

# Check the number of channels of the image
print('image channels:', image_rgb.shape[2])

Note: **uint8** means that we have a matrix of 8-bit unsigned integers 

### Generate a histogram of the color image
Sometimes you want to enhance the contrast in your image or expand the contrast in a particular region. A good tool to find interesting regions is the histogram, which shows a frequency distribution of the color values in an image. To create a histogram of our image data, we use the matplotlib.pyplot `hist()` function.

More info: [Histogram](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html#matplotlib.pyplot.hist)

Display the histogram of all the pixels in the color image with `plt.hist()`


In [ ]:
plt.hist(image.ravel(), bins = 256, range = [0,256])
plt.xlabel('value') 
plt.ylabel('count')
plt.title('eagle image histogram')
plt.show()

**Question:** 
What do you think the four peaks in the eagle histogram correspond to? 

**Response:**

####ToDo #2: Plot a histogram of your image `myImg_rgb` 

Be sure to add axis labels and a title 


In [ ]:
## plt.hist(myImg_rgb.ravel(),bins = 256, range = [0,256]) 
##
##
##
##

Now we'll display the histogram of each color channel (B, G, R)

In [ ]:
# Set colors for plotting each channel
color = ('b','g','r')
for i,col in enumerate(color):
    # calculate histogram
    image_hist = cv.calcHist([image], [i], None, [256], [0,256])
    plt.plot(image_hist, color = col)
    plt.xlim([0,256])
plt.xlabel('color value')
plt.ylabel('count')
plt.title('Histogram of each color channel')
plt.show()

Notice that the blue channel has a large peak at a value of around 90. This corresponds to the pixels that comprise the sky in the eagle image. 

####ToDo #3 Plot a histogram of each color for your image

In [ ]:
## Code in this cell

##Step 4: Grayscale 

In [ ]:
# Convert color image to grayscale
gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
cv2_imshow(gray_image)

In [ ]:
# Plot the histogram of the gray image
plt.hist(gray_image.ravel(),bins = 256, range = [0, 256])
plt.xlabel('value')
plt.ylabel('count')
plt.title('Grayscale histogram')
plt.show()

**Note:** The frequency of the image histogram has decreased by about 1/3 of the histogram of color image. 

We can see why by looking at the dimensions of the original color image and of the grayscale image. 

To do this we'll view the shape of the image matrix. The first two dimensions `(M, N)` correspond to the number of rows and columns of the image, respectively.  

In [ ]:
# Check the color image dimensions
print('color image dimensions:', image.shape)

# Check the grayscale image dimensions
print('grayscale image dimensions:', gray_image.shape)

#### ToDo #4: Display the grayscale of your image and generate its histogram


In [ ]:
## grayscale

In [ ]:
## grayscale histogram

# Thresholding 
Thresholding converts images to binary images, which we can then use to localize objects and obtain information about their size, shape, and position.

In [ ]:
## Helper function to plot multiple images
def plot_img(images, titles):
  fig, axs = plt.subplots(nrows = 1, ncols = len(images), figsize = (15, 15))
  for i, p in enumerate(images):
    axs[i].imshow(p, 'gray')
    axs[i].set_title(titles[i])
    #axs[i].axis('off')
  plt.show()

**Side Note:** Here we are using a different method to display images. We are calling the `plt.imshow()` method, which reads an `(M, N, 3)` image as RGB values rather than BGR like `cv2_imshow()`.

In [ ]:
# Download a useful image for the following sections of the tutorial

!wget https://raw.githubusercontent.com/adsoto/Shark-Analysis/main/zf-shoal-00004.jpg
 

To load an image using OpenCV, we can use the `imread()` function

In [ ]:
# Import the fish image 
im1 = cv.imread('zf-shoal-00004.jpg') 

In [ ]:
# Display the image
cv2_imshow(im1)

If this was a python script or Jupyter notebook running locally on your machine, you could read an image with a function call similar to the following:

`im = cv.imread('/image/path/image_name.jpg')`

In [ ]:
# Image dimensions
print('Original Dimensions : ', im1.shape)

This image is large, let's resize for efficiency and easier viewing

In [ ]:
# scale factor (0 to 1, where 1 returns the original size)
scale_factor = 0.5

# generate dimensions for resized image
width = int(im1.shape[1] * scale_factor)
height = int(im1.shape[0] * scale_factor)
dim = (width, height)
  
# resize image
im2 = cv.resize(im1, dim, interpolation = cv.INTER_AREA)
 
print('Resized Dimensions : ', im2.shape)

In [ ]:
# Display the resized image
cv2_imshow(im2)

## Simple Thresholding
If a pixel value is greater than the threshold value, it is assigned one value (may be white), else it is assigned another value (may be black). The function used is 
```
cv.threshold(img, thresh_value, maxVal, style)
```
*   First argument is the source image (typically a grayscale image).
*   Second argument is the threshold value which is used to classify the pixel values.
*   Third argument is the maxVal which represents the value to be given if pixel value is more than (sometimes less than) the threshold value. 
*   Third argument is the style of thresholding. OpenCV provides different styles of thresholding. We'll only try out the first two, but you can try the others yourself by providing the desired arugment. 

  1.  *cv.THRESH_BINARY*
  2.  *cv.THRESH_BINARY_INV*
  3.  *cv.THRESH_TRUNC*
  4.  *cv.THRESH_TOZERO*
  5.  *cv.THRESH_TOZERO_INV*

![alt text](https://miro.medium.com/max/730/1*swjBYQOnuNfv1rHM3p39PQ.png) 

**I(x, y)** is the intensity at the point, or the pixel value at (x, y).


Two outputs are obtained from the `cv.threshold()` function. The first one is a **retval**, the threshold used (or caluclated) by the thresholding method. Second output is the thresholded image.

###Binary

In [ ]:
## Binary Thresholding

# set the desired threshold value 
thresh_val = 70

# Convert to grayscale
im2_gray = cv.cvtColor(im2, cv.COLOR_BGR2GRAY)

# apply thresholding
ret, im2_binary = cv.threshold(im2_gray, thresh_val, 255, cv.THRESH_BINARY)

# Plot the images
images = [im2, im2_gray, im2_binary]
titles = ['Original image', 'Grayscale', 'THRESH_BINARY']
plot_img(images, titles)

#### ToDo #5: Modify the threshold value to describe its effect. 
Do this multiple times for **thresh_val** between 0 and 255. 

In [ ]:
# set the threshold value 
thresh_val = 30

# apply thresholding
ret, binary = cv.threshold(im2_gray, thresh_val, 255, cv.THRESH_BINARY)

# Plot the images
images = [im2_gray, binary]
titles = ['Graysclae Image', 'THRESH_BINARY']
plot_img(images, titles)

#### Describe effect of `thresh_val` here:

###Binary Inverse

In [ ]:
## Binary Inverse Thresholding
thresh_val = 80
ret, im_binary_inv = cv.threshold(im2_gray, thresh_val, 255, cv.THRESH_BINARY_INV)

# Plot the images
images = [im2_gray, im_binary_inv]
titles = ['Graysclae Image', 'THRESH_BINARY_INV']
plot_img(images, titles)

**Note:** This method could be very useful for blob analysis (next Section) on this image because the objects of interest are considered foreground (white) pixels. 

## Otsu’s Binarization
Otsu’s Binarization is used to perform automatic image thresholding. It automatically calculates a threshold value from the image histogram for a bimodal image. A bimodal image is an image with two distinct peaks (or humps) in its histogram.

We use the usual threshold function `cv.threshold()`, but pass an extra flag, *cv.THRESH_OTSU*. For threshold value, simply pass zero. Then the algorithm finds the optimal threshold value which is returned as `retVal`. If Otsu thresholding is not used, retVal is same as the threshold value you input.

In [ ]:
# global thresholding
ret_binary, img_binary = cv.threshold(im2_gray, 80, 255, cv.THRESH_BINARY)

# Otsu's thresholding
ret_otsu, img_otsu_binary = cv.threshold(im2_gray,0,255, cv.THRESH_BINARY+cv.THRESH_OTSU)

# Plot the images
images = [im2, img_binary, img_otsu_binary]
titles = ['Original image', 'THRESH_BINARY; retVal:' + str(ret_binary), 'THRESH_OTSU; retVal:' + str(ret_otsu)]
plot_img(images, titles)

As you can see, the automatic thresholding function doesn't guarantee better results. It depends a lot on the input image and what features you're interested in. 

##Adaptive thresholding
In Simple thresholding, we used a global value as threshold value. But it may not be good in all the conditions where image has different lighting conditions in different areas. In that case, we go for adaptive thresholding. In this, the algorithm calculate the threshold for a small regions of the image. So we get different thresholds for different regions of the same image and it gives us better results for images with varying illumination.
```
cv.adaptiveThreshold(img, maxValue, adaptiveMethod, thresholdType, blockSize, C)
```
It has three ‘special’ input params and only one output argument.

*   **Adaptive Method** :
  1. cv.ADAPTIVE_THRESH_MEAN_C : threshold value is the mean of neighbourhood area.
  2. cv.ADAPTIVE_THRESH_GAUSSIAN_C : threshold value is the weighted sum of neighbourhood values where weights are a gaussian window.
*   **Block Size** - It decides the size of neighbourhood area.
*   **C** - It is just a constant which is subtracted from the mean or weighted mean calculated.

In [ ]:
# Adaptive Threshold Mean C
im_thresh_mean_c = cv.adaptiveThreshold(im2_gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 11, 2)

# Plot the original, simple threshold, and adaptive threshold images
images = [im2_gray, im2_binary, im_thresh_mean_c]
titles = ['Graysclae Image', 'THRESH_BINARY', 'ADAPTIVE_THRESH_MEAN_C']
plot_img(images, titles)

In [ ]:
# Adaptive Threshold Mean C
im_thresh_gauss_c = cv.adaptiveThreshold(im2_gray, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)

# Plot the original, simple threshold, and adaptive threshold images
images = [im2_gray, im_thresh_mean_c, im_thresh_gauss_c]
titles = ['Graysclae Image', 'ADAPTIVE_THRESH_MEAN_C', 'ADAPTIVE_THRESH_GAUSSIAN_C']
plot_img(images, titles)

#### ToDo #6

Based on what you observe in the images above, describe the effect that the Gaussian_C method has on the thresholding compared to the Mean_C method. 

#### Describe difference here:

## Thresholding a color image
> I mentioned in the Simple Thresholding section that we typically apply thresholding to grayscale images. So is it possible to threshold a color image? 

Let's try it!




In [ ]:
# apply thresholding directly to color image of eagle
ret, eagle_binary = cv.threshold(image_rgb, thresh_val, 255, cv.THRESH_BINARY)

# Plot the images
images_1 = [image, eagle_binary]
titles_1 = ['Original image', 'Threshold (color)']
plot_img(images_1, titles_1)

In [ ]:
# apply thresholding to graysale image of eagle
ret, eagle_gray_binary = cv.threshold(gray_image, 100, 255, cv.THRESH_BINARY)

images_2 = [gray_image, eagle_gray_binary]
titles_2 = ['Grayscale', 'Threshold (gray)']
plot_img(images_2, titles_2)

Thresholding on color channels is possible and may be a useful way of isolating image features within a particualr color range. 

# Blob analysis
This section makes use of our binary images to find connected components, or **blobs** in the image - usually the white pixels in a binary image. 

## Image Contours

A contour is the set of points joining all the continuous points (along the boundary) that have same color or intensity. This is another name for **blob**

There are three arguments in the `cv.findContours(img, mode, method)` function. The first one is the source image, second is contour retrieval mode, third is contour approximation method. 

It outputs the contours and hierarchy. *contours* is a list of all the contours in the image. Each individual contour is a Numpy array of (x,y) coordinates for the boundary points of the blob.

Here we'll apply `cv.findContours()` to `im2_binary` from the previous section.

In [ ]:
# find the image contours
contours, hierarchy = cv.findContours(im2_binary, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

In [ ]:
print("Number of Contours found = " + str(len(contours)))

To visualize the contours we'll use `cv.drawContours()`, 

Its first argument is the source image, second argument is the contours which should be passed as a Python list, third argument is the index of contours (useful when drawing an individual contour -- to draw all contours, pass -1) and remaining arguments are color and thickness.

We'll create a copy of the `im2` image so that we can draw the contours and preserve the original.

In [ ]:
# Copy the image to preserve original
im2_copy = im2.copy()

# overlay the contours on the original image 
im2_overlay = cv.drawContours(im2_copy, contours, -1, (0,255,0), 1)

# Plot the image with overlayed contours 
cv2_imshow(im2_overlay)

### Sort contours by area

Let's draw only the first 10 contours to visualize where they are.

In [ ]:
# Copy the image to preserve original
im2_copy = im2.copy()

# Draw the first 10 contours from the unsorted list
im2_unsorted = cv.drawContours(im2_copy, contours[0:9], -1, (0,255,0), 2)

# Plot the image with overlayed contours 
cv2_imshow(im2_unsorted)

Do you see the green contours?
> They are very small and aren't features we're interested in. What we're after are the contours that outline the fish. So we'll sort the contours list, hopefully they are the largest contours in the image. 

In [ ]:
# largest to smallest sorted list
sorted_contours = sorted(contours, key = cv.contourArea, reverse = True)

In [ ]:
# Copy the image to preserve original
im2_copy = im2.copy()

# Draw the first 10 contours from the unsorted list
im2_sorted = cv.drawContours(im2_copy, sorted_contours[0:9], -1, (0,255,0), 2)

# Plot the image with overlayed contours 
cv2_imshow(im2_sorted)

So the fish are not the largest contours, but they are among the largest. Now we can use information about their area to be more selective as we extract properties of the fish blobs. 

## Extract blob properties

In [ ]:
# Function to find some useful properties for sorting and localizing objects

def blob_properties(contours):

  # initialize list to keep track of blob properties
  cont_props = []
  i = 0

  # loop through each contour to find their properties
  for cnt in contours:

    # compute contour moments
    M = cv.moments(cnt)

    # compute centroid of contour
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    area = cv.contourArea(cnt)

    # Add parameters to list
    add = [i, (cx, cy), area]
    cont_props.append(add)
    i += 1

  return cont_props

Let's call `blob_properties()` to get the centroid position and area of the first 10 contours of our sorted list. 

In [ ]:
cont_props = blob_properties(sorted_contours[0:10])

In [ ]:
cont_props

# Lab 2: Homework
Create a new Google Colab notebook with the title **lab_2_hw_your_initials** for this assignment. 

Using one of the images from your ball throwing sequence (choose one where the ball is clearly visible), do the following:

> 1. Plot the full image histogram
1. Plot the histogram of each color channel
1. Convert the image to grayscale and display its histogram
1. Apply a thresholding technique of your choice such that the we clearly see the ball blob.
1. Plot the original, grayscale, and binary images together (side-by-side)
1. Compute the image contours and visualize them 
1. Extract and print the centroid position of the ball blob and its area

You will submit a pdf of the complete notebook (try: File-->Print-->Save as PDF) as well as the notebook itself (File-->Download-->Download .ipynb).

**Note:** Every student will submit their own pdf (on gradescope) and notebook (on Google Drive). 

## File upload code to import an image from your hard drive. 
Here's a code snippet that will be useful for the homework. 

Run the cell and then clink `Choose Files`

In [ ]:
# Import one of your ball throwing images 

uploaded = files.upload()
for filepath in uploaded.keys():
  print(f'User uploaded file "{filepath}"')

img_filepath = os.path.abspath(filepath)
img_ball = cv.imread(img_filepath) 

# Sources
This notebook was adapted from Prof Soto's notes and the following sources:

[source 1](https://docs.opencv.org/4.x/index.html)

[source 2](https://github.com/xn2333/OpenCV/blob/master/Seminar_Image_Processing_in_Python.ipynb)

[source 3](https://code.adonline.id.au/blob-analysis-with-opencv-in-python/)

[source 4](https://learnopencv.com/blob-detection-using-opencv-python-c/)

## Extras (not needed)

[source](https://learnopencv.com/blob-detection-using-opencv-python-c/)

[source](https://rubikscode.net/2022/06/13/thresholding-edge-contour-and-line-detection-with-opencv/)

In [ ]:
# Function to find some useful properties for sorting and localizing objects

# def blob_properties_full(contours):

#   # initialize list to keep track of blob properties
#   cont_props= []
#   i = 0

#   # loop through each contour to find their properties
#   for cnt in contours:

#     # compute contour moments
#     M = cv.moments(cnt)

#     # compute centroid of contour
#     cx = int(M['m10']/M['m00'])
#     cy = int(M['m01']/M['m00'])

#     area = cv.contourArea(cnt)
#     perimeter = cv.arcLength(cnt,True)

#     # bounding rectangle
#     x1,y1,w,h = cv.boundingRect(cnt)
#     x2 = x1+w
#     y2 = y1+h
#     aspect_ratio = float(w)/h

#     # convex hull and hull area
#     hull = cv.convexHull(cnt)
#     hull_area = cv.contourArea(hull)

#     rect = cv.minAreaRect(cnt)
#     ellipse = cv.fitEllipse(cnt)

#     # Add parameters to list
#     add = [i, cx, cy, area, round(perimeter, 1), round(aspect_ratio, 3), w, h, round(hull_area, 1), x1, y1, x2, y2, rect, ellipse]
#     cont_props.append(add)
#     i += 1

#   return cont_props